# CRF算子 (CRF Operator)

This CRF operator is simpler than the implementation on PyTorch (https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html?highlight=ner), because I don't use START_TAG and STOP_TAG. So you can focus more on the actual tags you are trying to predict.

* mainly used for NER problem
* In: emission score matrix
    * size of row: count of words in a sentence decided by the length of sentence (no need to consider);
    * size of column: the target entity type sequence (set through tag_size during initialization)
* Out: tuple (best_score, best_path)
    * *best_score* refers to the score calculated according to CRF algorithm
    * *best_path* refers to the predicted types of entity, which is a sequence
* Loss function for training: *loss_nll_crf(emission, tags)*
    * This self-defined loss function (specifically for CRF model) will be called during training process

If you have any troubleshooting using this operator, DO let me know. Thank you!

In [1]:
import torch
import torch.nn as nn
import numpy as np

# torch.manual_seed(3407) is all you need: On the influence of random seeds in DL architectures for computer vision
# Well, I just set 3407 for fun, nothing serious ...
torch.manual_seed(3407)


class CRF(nn.Module):

    def __init__(self, tag_size):
        super(CRF, self).__init__()

        # real size of tags from input
        self.tag_size = tag_size

        # PARAM: transitional matrix - element (i,j) refers to the score transitioning from i to j
        self.transition = nn.Parameter(
            data=torch.tensor(data=np.random.randn(tag_size, tag_size), dtype=torch.float64)
        )

    def _score_real_path(self, emission, tags):
        """
        A capsuled internal logic for forward() API. DO NOT call this function externally!!!
        This function is used for calculating the score of real path given by dataset
        :param emission: emission matrix, size: (seq_len, tag_size)
        :param tags: the given tags in the dataset, size: (seq_len), where tags[i] represents word_i's real tag
        :return: current score of the given real path
        """
        score = emission[0, tags[0]]
        for i, cur_emission in enumerate(emission[1:], start=1):
            score = score + cur_emission[tags[i]] + self.transition[tags[i - 1], tags[i]]
        return score

    def _score_all_paths(self, emission):
        """
        A capsuled internal logic for forward() API. DO NOT call this function externally!!!
        This function is used for calculating the TOTAL score of all possible combinations of tags in a given sequence
        :param emission: emission matrix, size: (seq_len, tag_size)
        :return: TOTAL score of all the possible combinations of tags
        """
        pre_score = torch.zeros(size=[1],
                                device=f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu") + \
                    emission[0]

        for i, cur_emit in enumerate(emission[1:], start=1):
            cur_score = pre_score.unsqueeze(dim=1) + cur_emit + self.transition
            pre_score = torch.log(torch.sum(torch.exp(cur_score), dim=0))
        # then calculate all the elements in pre_score
        pre_score = torch.log(torch.sum(torch.exp(pre_score)))
        return pre_score

    def _viterbi_decode(self, emission):
        """
        A capsuled internal logic for forward() API. DO NOT call this function externally!!!
        Given the emission score of input sequence, find out the best tag sequence (its score & best path)
        Algorithm: viterbi decoding
        :param emission: emission score matrix, size: (seq_len, tag_size)
        else size: (batch_size, seq_len, tag_size)
        :return: (score, tag_seq) - score of the best path, size: (tag_size); and corresponding tag_seq, size: (tag_size)
        """
        # init: The original score is only the score of word0 in emission score
        best_scores = torch.clone(emission[0, :])
        # init: The previous tag index of current index, where the score of the path to current tag is the best
        previous = torch.zeros(size=emission.shape, dtype=torch.int)
        previous[0] = -1

        for i, cur_emit in enumerate(emission[1:], start=1):
            cur_score = best_scores.unsqueeze(dim=1) + cur_emit + self.transition
            best_scores, previous_idx = torch.max(cur_score, dim=0)
            # best_scores can be updated automatically, but max_previous should be preserved
            previous[i] = previous_idx

        # for the final status of best_scores, grab the max score and its current index
        best_score, best_idx = best_scores.max(dim=0)
        best_score = best_score.tolist()
        best_idx = best_idx.tolist()
        # find out the best path from prevoius
        best_path = [best_idx]
        for i in range(previous.shape[0] - 1, 0, -1):
            best_idx = previous[i, best_idx].item()
            best_path.append(best_idx)
        # as a stack, reverse the best_path
        best_path = best_path[::-1]
        return best_score, best_path

    def forward(self, emissions, batch_valid_lens=None, return_tensor=True):
        """
        Given the emission score of input sequence, find out the best tag sequence (its score & best path)
        Algorithm: viterbi decoding
        :param emissions: emission score matrix, size: (seq_len, tag_size) if batch_valid_lens is not given,
        else size: (batch_size, seq_len, tag_size)
        :param batch_valid_lens: (optional) valid length of each sentence given in a batch, default None
        :return: (score, tag_seq) - score of the best path, size: (tag_size); and corresponding tag_seq, size: (tag_size)
        :param return_tensor: If True, the operator will return tensor formed data. Otherwise, return traditional python numeric type and a list type. default: True
        """
        if batch_valid_lens is None:
            assert emissions.dim() == 2
            best_scores, best_paths = self._viterbi_decode(emissions)
        else:  # use batch
            assert len(emissions) == len(batch_valid_lens)
            best_scores, best_paths = [], []
            # eliminate unwanted paddings before decoding
            for i, emission in enumerate(emissions):
                emit = emission[:batch_valid_lens[i], :]
                best_score, best_path = self._viterbi_decode(emit)
                best_scores.append(best_score)
                best_paths.append(best_path)

        if return_tensor:
            best_scores = torch.tensor(best_scores)
            if type(best_paths[0]) == list:
                best_paths = [torch.tensor(bp) for bp in best_paths]
            else:
                best_paths = torch.tensor(best_paths, dtype=torch.int)
        else:
            pass
        return best_scores, best_paths

    def loss_nll_crf(self, emissions, tags):
        """
        Self-defined loss function specifically for CRF. (Negtive Log Likelihood)
        Normally, the loss function should be integrated with other DL semantic encoding layer.
        :param emissions: given emission score from
        :param tags: given real tags of the sentence from dataset
        :return: Loss of the CRF = log(sum(exp(all_the_paths))) - score_of_real_path
        """
        return self._score_all_paths(emissions) - self._score_real_path(emissions, tags)

Experiment

In [2]:
# set up device

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda:0 device


In [3]:
# For CRF, normally, X refers to the emission matrix.
# For demo purpose, here I initialize it randomly.
# But you should use careful feature engineering method OR find out emission matrix from other DL model
X = torch.randn(size=[8, 5]).to(device)
X.shape[1]

5

In [4]:
crf_op = CRF(tag_size=X.shape[1]).to(device=device)

In [5]:
with torch.no_grad():
    best_score, best_path = crf_op(X)

In [6]:
best_score, best_path

(tensor(13.4126), tensor([3, 4, 3, 2, 2, 2, 2, 4], dtype=torch.int32))

a simple reference for showing how to train the model with batched data

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# specify your device (CUDA supported)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"

print(f"Using {device} device")

# make up a group of batch data as some toy data (you should use real data during training process)
valid_lens = [8, 4, 5]
training_data = [
    ((torch.randn(size=[3, 8, 5]).to(device), torch.tensor(valid_lens)),
     torch.tensor([[1, 2, 0, 3, 2, 4, 3, 0], [1, 4, 2, 3, 5, 5, 5, 5], [0, 1, 3, 2, 4, 5, 5, 5]])
     )
]

# build the model
crf_op = CRF(tag_size=5).to(device=device)

# create an optimizer
optimizer = optim.SGD(crf_op.parameters(), lr=0.01, weight_decay=1e-6)

# train the model
# normally you would NOT do 5000 epochs, This is only toy data!!!
for epoch in range(15000):
    for sentence_idx, tags in training_data:
        emissions, valid_lengths = sentence_idx
        # loss: use self-defined nll loss function for CRF model
        for i, emit in enumerate(emissions):
            loss = crf_op.loss_nll_crf(emit[:valid_lengths[i], :], tags[i][:valid_lengths[i]])
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

Using cuda:0 device


In [8]:
# Check predictions after training
with torch.no_grad():
    score, labels = crf_op(training_data[0][0][0], batch_valid_lens=training_data[0][0][1])
    print(score)
    print(labels)

tensor([29.1700, 14.3275, 20.0263])
[tensor([2, 4, 3, 2, 3, 2, 4, 2]), tensor([1, 2, 0, 3]), tensor([1, 3, 0, 1, 4])]


If you don't train batched data, here is the reference:

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

# specify your device (CUDA supported)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"

print(f"Using {device} device")

# make up some toy data (you should use real data during training process)
X = torch.randn(size=[8, 5]).to(device)
y = [1, 2, 0, 3, 2, 4, 3, 0]
training_data = [
    (X, y)
]

# build the model
crf_op = CRF(tag_size=5).to(device=device)

# create an optimizer
optimizer = optim.SGD(crf_op.parameters(), lr=0.01, weight_decay=1e-6)

# train the model
# normally you would NOT do 5000 epochs, This is only toy data!!!
for epoch in range(5000):
    for emission, tags in training_data:
        # loss: use self-defined nll loss function for CRF model
        loss = crf_op.loss_nll_crf(emission, tags)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Check predictions after training
with torch.no_grad():
    score, labels = crf_op(training_data[0][0])
    print(score)
    print(labels)

Using cuda:0 device
tensor(42.4312)
tensor([1, 2, 4, 3, 0, 3, 2, 0], dtype=torch.int32)
